<a href="https://colab.research.google.com/github/ilona-krasnova/IRM3006/blob/main/IRM3006P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing libraries in Python
import pandas as pd
import numpy as np
import re
import json

# install playwright
!pip install -q playwright
!playwright install

import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 9.6 MB/s eta 0:00:00
164.5 MiB [] 0% 0.0s164.5 MiB [] 0% 40.3s164.5 MiB [] 0% 27.4s164.5 MiB [] 0% 14.1s164.5 MiB [] 0% 9.4s164.5 MiB [] 1% 7.0s164.5 MiB [] 1% 6.2s164.5 MiB [] 2% 5.7s164.5 MiB [] 2% 5.6s164.5 MiB [] 2% 5.7s164.5 MiB [] 3% 5.4s164.5 MiB [] 3% 5.1s164.5 MiB [] 3% 5.3s164.5 MiB [] 4% 5.2s164.5 MiB [] 4% 5.4s164.5 MiB [] 4% 5.3s164.5 MiB [] 5% 5.1s164.5 MiB [] 5% 4.7s164.5 MiB [] 6% 4.6s164.5 MiB [] 6% 4.7s164.5 MiB [] 7% 4.2s164.5 MiB [] 8% 4.1s164.5 MiB [] 9% 4.0s164.5 MiB [] 9% 3.8s164.5 MiB [] 10% 3.8s164.5 MiB [] 11% 3.6s164.5 MiB [] 12% 3.5s164.5 MiB [] 13% 3.4s164.5 MiB [] 14% 3.3s164.5 MiB [] 15% 3.3s164.5 MiB [] 16% 3.3s164.5 MiB [] 17% 3.2s164.5 MiB [] 17% 3.1s164.5 MiB [] 18% 3.0s164.5 MiB [] 19% 3.0s164.5 MiB [] 20% 2.9s164.5 MiB [] 20% 2.8s164.5 MiB [] 21% 2.8s164.5 MiB [] 22% 2.7s164.5 MiB [] 23% 2.6s164.5 MiB [] 24% 2.6s164.5 MiB [] 25% 2.5s164.5 MiB [] 26% 2.4s164.5 MiB [] 28% 2.3s164.5 MiB [] 29% 2.

In [12]:
# Read search page and save to file

from playwright.async_api import Page

async def fetch_dynamic_content(url, browser):
    # Output "0" to console
    # print(0)

    # Create a new page within the existing browser context
    page = await browser.new_page()
    await page.goto(url)

    # Output "1" to console
    # print(1)

    # Wait for the dynamic content to load
    # await page.wait_for_selector('.result-item-text') # maybe find a better benchmark for page load?
    await page.wait_for_load_state('networkidle')

    # Output "2" to console
    # print(2)

    # Extract page content after rendering
    page_content = await page.content()

    # Write page content to "search_results.txt"
    with open("search_results.txt", "w") as f:
        f.write(page_content)

    # Close the page (but not the browser)
    await page.close()

    # Return the page content as a result
    return page_content

In [13]:
# Process search results; output all article links

def get_article_links(html_content):
    soup = BeautifulSoup(html_content, "html.parser")

    # Extract href values and store them in an array
    href_values = [a['href'] for a in soup.select("h3.item-title > a") if 'href' in a.attrs]

    # `href_values` now contains all the href attributes as an array
    return href_values

# Load the content of `search_results.txt` into BeautifulSoup
# with open("search_results.txt", "r") as f:
#     html_content = f.read()

# href_values2 = get_article_links(html_content)

# # Print the array to the console with each entry on its own line
# for href in href_values2:
#     print(href)

In [14]:
def find_dates_in_text(text):
    # If the text doesn't exist, return
    if not text:
        return []
    # Find all sequences of exactly 4 digits
    four_digit_numbers = re.findall(r'\d{4}', text)
    # Filter numbers within the range 1801 to 2025 and remove duplicates by converting to a set
    dates = {int(num) for num in four_digit_numbers if 1801 <= int(num) <= 2025}
    # Sort the unique dates in descending order
    return sorted(dates, reverse=True)

In [15]:
# Process array of strings to remove duplicates and filter out unwanted values.
def filter_and_deduplicate_strings(strings):
    if not strings:
        return []
    unwanted_values = {'more', 'hide', 'Show All', 'Show Less', ''}
    # Remove duplicates, strip whitespace, and filter out unwanted values
    unique_strings = list({string.strip() for string in strings})
    filtered_strings = [text for text in unique_strings if text not in unwanted_values]
    return filtered_strings

In [16]:
def read_details_segment(soup, segment_name):
    # Parse the HTML content with BeautifulSoup
    # soup = BeautifulSoup(content, 'html.parser')

    # Step 1: Find the span with the attribute data-details-label="title"
    title_span = soup.find('span', {'data-details-label': segment_name})

    if not title_span:
        print("No span with data-details-label='" + segment_name +"' found")
        return

    # Step 2: Find the div that is the parent of this span
    parent_div = title_span.find_parent('div')

    if not parent_div:
        print("No parent div found for the " + segment_name +" span")
        return

    # Step 3: Find the div that is the next sibling of the parent div
    next_div = parent_div.find_next_sibling('div')

    if not next_div:
        print("No next sibling div found for the parent div (" + segment_name + ")")
        return

    # Step 4: In this next div, find the span and get its contents
    target_span = next_div.find_all('span')

    # Step 5: Output the contents of the target span
    if not target_span:
        print("No spans found in the next div for " + segment_name)
        return []
    return [span.get_text() for span in target_span]

def read_details(content, year):
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')

    # Usage example with content
    title = read_details_segment(soup, 'title')
    # print(title[0])

    creator = filter_and_deduplicate_strings(read_details_segment(soup, 'creator'))
    # print(creator)

    ispartof = read_details_segment(soup, 'ispartof')
    # print(ispartof[0])

    publisher = read_details_segment(soup, 'publisher')
    # print(publisher[0])

    subject = filter_and_deduplicate_strings(read_details_segment(soup, 'subject'))
    # print(subject)

    description = read_details_segment(soup, 'description')
    # print(description[0])

    identifier = filter_and_deduplicate_strings(read_details_segment(soup, 'identifier'))
    # print(identifier)

    # dates = find_dates_in_text(ispartof[0])

    # Construct JSON string with title, creator, etc.

    # Define your data as a dictionary
    json_data = {
        "title": title[0] if title else "",
        "creator": creator,
        "date": year,
        "ispartof": ispartof[0] if ispartof else "",
        "publisher": publisher[0] if publisher else "",
        "subject": subject,
        "description": description[0] if description else "",
        "identifier": identifier
    }

    # Convert the dictionary to a JSON string if needed
    json_string = json.dumps(json_data, ensure_ascii=False)

    print(json_string)

    # Append the JSON string to a text file
    with open("bibliometrics.txt", "a", encoding="utf-8") as file:
        file.write(json_string + "\n")  # Add newline for each JSON entry

# Main ♟

In [19]:
async def main():
    async with async_playwright() as p:
        # Launch the browser once
        browser = await p.chromium.launch(headless=True)

        # Prompt user to input URL (OMNI Search)
        # removed last "0" from url to avoid having to remove it systematically
        base_url = "https://ocul-crl.primo.exlibrisgroup.com/discovery/search?query=any%2Ccontains%2Cbombus+terrestris&tab=OCULDiscoveryNetworkNew&search_scope=NewDiscoveryNetwork&vid=01OCUL_CRL%3ACRL_DEFAULT&mfacet=rtype%2Cinclude%2Carticles%2C1&mfacet=searchcreationdate%2Cinclude%2C1980%7C%2C%7C1980%2C2&mode=basic&offset="

        # Truncate end of base_url after last "="
        truncate_position = base_url.rfind("=") + 1
        base_url = base_url[:truncate_position]

        # # Prompt user to input first page number
        # first_page = int(input("Enter first page number: "))

        # # Prompt user to input last page number
        # last_page = int(input("Enter last page number (incl.): "))

        # Prompt user to input first year
        first_year = int(input("Enter first year: "))

        # Prompt user to input last year
        last_year = int(input("Enter last year: "))

        # For each year
        for year in range(first_year, last_year + 1):
            print("Year ", year)
            # replace "1980" in url with year
            year_url = base_url.replace("1980", str(year))

            # For each search result page
            for page in range(0, 25):
                url = f"{year_url}{page*10}"
                print("Page ", page + 1)

                # Parse search results
                search_results = await fetch_dynamic_content(url, browser)

                # Parse HTML
                links = get_article_links(search_results)
                if not links:
                    break

                # For each article
                for link in links:
                    print(link)
                    content = await fetch_dynamic_content(link, browser)
                    read_details(content, year)

        # # Fetch content from multiple URLs
        # content1 = await fetch_dynamic_content("https://ocul-crl.primo.exlibrisgroup.com/discovery/fulldisplay?docid=cdi_proquest_journals_2938293070&context=PC&vid=01OCUL_CRL:CRL_DEFAULT&lang=en&search_scope=NewDiscoveryNetwork&adaptor=Primo", browser)
        # content2 = await fetch_dynamic_content("https://ocul-crl.primo.exlibrisgroup.com/discovery/fulldisplay?docid=cdi_proquest_journals_2913337499&context=PC&vid=01OCUL_CRL:CRL_DEFAULT&lang=en&search_scope=NewDiscoveryNetwork&adaptor=Primo", browser)

        # # save content1 to file
        # with open("content1.txt", "w") as f:
        #     f.write(content1)

        # # save content2 to file
        # with open("content2.txt", "w") as f:
        #     f.write(content2)

        # Close the browser after all calls
        await browser.close()

# run main function
await main()

Enter first year: 1900
Enter last year: 1989
Year  1900
Page  1
Year  1901
Page  1
Year  1902
Page  1
Year  1903
Page  1
Year  1904
Page  1
Year  1905
Page  1
Year  1906
Page  1
Year  1907
Page  1
Year  1908
Page  1
Year  1909
Page  1
Year  1910
Page  1
Year  1911
Page  1
Year  1912
Page  1
Year  1913
Page  1
Year  1914
Page  1
Year  1915
Page  1
Year  1916
Page  1
Year  1917
Page  1
Year  1918
Page  1
https://ocul-crl.primo.exlibrisgroup.com/discovery/fulldisplay?docid=cdi_smithsonian_biodiversity_v2_oai_biodiversitylibrary_org_part_225481&context=PC&vid=01OCUL_CRL:CRL_DEFAULT&lang=en&search_scope=NewDiscoveryNetwork&adaptor=Primo Central&tab=OCULDiscoveryNetworkNew&query=any%2Ccontains%2Cbombus%20terrestris&mode=basic&offset=0
No span with data-details-label='creator' found
No span with data-details-label='publisher' found
No span with data-details-label='subject' found
No span with data-details-label='description' found
No span with data-details-label='identifier' found
{"title": "B

In [ ]:
url = "https://ocul-crl.primo.exlibrisgroup.com/discovery/search?query=any,contains,bombus%20terrestris&tab=OCULDiscoveryNetworkNew&search_scope=NewDiscoveryNetwork&vid=01OCUL_CRL:CRL_DEFAULT&mfacet=rtype,include,articles,1&mode=basic&offset=0"



https://ocul-crl.primo.exlibrisgroup.com/discovery/search?query=any,contains,bombus%20terrestris&tab=OCULDiscoveryNetworkNew&search_scope=NewDiscoveryNetwork&vid=01OCUL_CRL:CRL_DEFAULT&mfacet=rtype,include,articles,1&mode=basic&offset=
